# Notebook: Train Model for a given Condition

## Packages

In [1]:
from ACD import aspect_category_labels_to_one_hot, CustomDatasetACD, preprocess_data_ACD, create_model_ACD, compute_metrics_ACD, get_trainer_ACD
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, precision_score, recall_score
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Dataset as TorchDataset
from helper import format_seconds_to_time_string
from transformers import DataCollatorWithPadding
from transformers import set_seed
from scipy.special import expit
import pandas as pd
import numpy as np
import datetime
import warnings
import random
import torch
import json
import time

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.optimization")
torch.device("mps")

device(type='mps')

## Parameters

In [3]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "aspect_term" # "aspect_term", "aspect_category"
LLM_SAMPLING = "fixed"

## Settings (do not change!)

In [4]:
N_FOLDS = 5

In [5]:
SPLIT_LOOP = [0, 1, 2, 3, 4, 0, 1, 2, 3]

In [6]:
RANDOM_SEED = 43
random.seed(RANDOM_SEED)

In [7]:
ASPECT_CATEGORIES  = ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]
POLARITIES = ["POSITIVE", "NEUTRAL", "NEGATIVE"]

In [8]:
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)

In [9]:
set_seed(RANDOM_SEED)

## Code

### Load Dataset

In [10]:
# Load Real Dataset
splits_real = []
for i in range(N_FOLDS):
    with open(f'../03 dataset split/real/real_{i}.json', 'r') as json_datei:
        real_split = json.load(json_datei)[:N_REAL]
        splits_real.append(real_split)

In [11]:
# Load Synth Dataset
splits_synth = []
for i in range(N_FOLDS):
    with open(f'../04 llm synthesis/synth/{LLM_NAME}/{LLM_SAMPLING}/split_{i}.json', 'r') as json_datei:
        synth_split = json.load(json_datei)[:N_SYNTH]
        splits_synth.append(synth_split)

In [12]:
n_splits_map = {
    500: 1,
    1000: 2,
    2000: 4
}
n_splits_required_real = n_splits_map.get(N_REAL, 0)
n_splits_required_synth = n_splits_map.get(N_SYNTH, 0)
n_splits_required_real, n_splits_required_synth

(1, 0)

In [13]:
# Five indexes, each for one cross valdiation run
train_dataset = []
test_dataset = []

for i in range(N_FOLDS):
    test_data = splits_real[i]
    train_data = []
    
    if N_REAL > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_real]:
            for example in splits_real[split_idx]:
                train_data.append(example)
 
    if N_SYNTH > 0:
        for split_idx in SPLIT_LOOP[i+1: i+1+n_splits_required_synth]:
            for example in splits_synth[split_idx]:
                train_data.append(example)
                
    random.shuffle(train_data)
    
    train_dataset.append(train_data)
    test_dataset.append(test_data)

### ACD Model

In [14]:
def train_ACD_model():
    results = {
        "LLM_NAME": LLM_NAME,
        "N_REAL": N_REAL,
        "N_SYNTH": N_SYNTH,
        "TARGET": TARGET,
        "LLM_SAMPLING": LLM_SAMPLING,
    }

    f1_micro_scores = []
    f1_macro_scores = []
    f1_weighted_scores = []
    accuracy_scores = []
    class_f1_scores = []
    loss = []
    hamming = []

    start_time = time.time()

    tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-large")
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    for cross_idx in range(N_FOLDS):
        # Load Data
        train_data = preprocess_data_ACD(train_dataset[cross_idx], tokenizer)
        test_data = preprocess_data_ACD(test_dataset[cross_idx], tokenizer)

        # Load Model
        model_ACD = create_model_ACD()

        # Train Model
        trainer = get_trainer_ACD(model_ACD, train_data, test_data, tokenizer)
        trainer.train()

        # Save Evaluation of Test Data
        eval_metrics = trainer.evaluate()

        # Save Metrics for fold
        f1_micro_scores.append(eval_metrics["eval_f1_micro"])
        f1_macro_scores.append(eval_metrics["eval_f1_macro"])
        f1_weighted_scores.append(eval_metrics["eval_f1_weighted"])
        accuracy_scores.append(eval_metrics["eval_accuracy"])
        class_f1_scores.append(eval_metrics["eval_class_f1_scores"])
        loss.append(eval_metrics["eval_loss"])
        hamming.append(eval_metrics["eval_hamming_loss"])

    runtime = time.time() - start_time

    results["loss"] = np.mean(loss)
    results["hamming"] = np.mean(hamming)
    results["accuracy"] = np.mean(accuracy_scores)
    results["f1_micro"] = np.mean(f1_micro_scores)
    results["f1_macro"] = np.mean(f1_macro_scores)
    results["f1_weighted"] = np.mean(f1_weighted_scores)
    results["runtime"] = runtime
    results["runtime_formatted"] = format_seconds_to_time_string(runtime)
    return results

if TARGET == "aspect_category":
   results = train_ACD_model()

### OTE Model

In [15]:
for cross_idx in range(N_FOLDS)[0:]:
    # Load Data
    train_data = train_dataset[cross_idx]
    test_data = test_dataset[cross_idx]

In [16]:
label2id = {
 'O': 0,
 'B': 1,
 'I': 2,
}
id2label = {
  0: 'O',
  1: 'B',
  2: 'I',
}

In [17]:
n_labels = len(id2label)

In [18]:
def dataset_to_aspect_level_OTE(dataset):
    aspect_dataset = []
    for example in dataset:
        aspect_categories = list(set([tag["label"] for tag in example["tags"] if tag["type"] == "label-explicit"]))
        for ac in aspect_categories:
            aspect_dataset.append({
                "text": example["text"],
                "aspect_category": ac,
                "tags": [tag for tag in example["tags"] if tag["type"] == "label-explicit"],
                "id": example["id"]
            })
    return aspect_dataset

In [19]:
def get_token_role_in_span_OTE(token_start: int, token_end: int, span_start: int, span_end: int):
    if token_end <= token_start:
        return "N"
    if token_start < span_start or token_end > span_end:
        return "O"
    if token_start > span_start:
        return "I"
    else:
        return "B"

In [20]:
def preprocess_example_OTE(example, tokenizer):
    input_text = example["text"] + "[SEP]" + example["aspect_category"]
    one_hot_output = [[0 for _ in label2id.keys()] for _ in range(256)]

    tokenized_input_text = tokenizer(input_text, 
                             return_offsets_mapping=True, 
                             padding="max_length", 
                             max_length=256,
                             truncation=True)

    for (token_start, token_end), token_labels in zip(tokenized_input_text["offset_mapping"], one_hot_output):
        for span in example["tags"]:
            role = get_token_role_in_span_OTE(token_start, token_end, span["start"], span["end"])
            if role == "B":
                token_labels[label2id["B"]] = 1
            elif role == "I":
                token_labels[label2id["I"]] = 1

    # 'input_ids', 'attention_mask', 'offset_mapping', 'labels'
    return {
        "input_ids": tokenized_input_text["input_ids"], 
        "attention_mask": tokenized_input_text["attention_mask"],
        "offset_mapping": tokenized_input_text["offset_mapping"],
        "labels": one_hot_output
    }

In [21]:
class CustomDatasetOTE(TorchDataset):
    def __init__(self, input_ids, attention_mask, offset_mapping, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.offset_mapping = offset_mapping
        self.labels = labels

    def __getitem__(self, idx):
        #item = {key: val[idx].clone().detach() for key, val in self.input_ids.items()}
        item = {}
        item["input_ids"] = self.input_ids[idx]
        item["attention_mask"] = self.attention_mask[idx]
        item["offset_mapping"] = self.offset_mapping[idx]
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

In [22]:
train_data = dataset_to_aspect_level_OTE(train_data)
test_data = dataset_to_aspect_level_OTE(test_data)

In [23]:
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-large")

train_data = [preprocess_example_OTE(example, tokenizer) for example in train_data]
test_data = [preprocess_example_OTE(example, tokenizer) for example in test_data]

In [24]:
train_data[0].keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'labels'])

In [25]:
train_data = CustomDatasetOTE([example["input_ids"] for example in train_data], 
                              [example["attention_mask"] for example in train_data],
                              [example["offset_mapping"] for example in train_data],
                              [example["labels"] for example in train_data])
test_data = CustomDatasetOTE([example["input_ids"] for example in test_data], 
                              [example["attention_mask"] for example in test_data],
                              [example["offset_mapping"] for example in test_data],
                              [example["labels"] for example in test_data])

In [26]:
batch_size = 16
epochs = 1
learning_rate = 5e-06

training_args = TrainingArguments(
    output_dir="output2",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="logs",
    logging_steps=100,
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
    fp16=torch.cuda.is_available(),
    report_to="none",
    seed=RANDOM_SEED,
)

In [27]:
def divide(a: int, b: int):
    return a / b if b > 0 else 0

def compute_metrics_OTE(p):
    predictions, true_labels = p
    predicted_labels = np.where(predictions > 0, np.ones(predictions.shape), np.zeros(predictions.shape))
    metrics = {}
    cm = multilabel_confusion_matrix(true_labels.reshape(-1, n_labels), predicted_labels.reshape(-1, n_labels))
    
    for label_idx, matrix in enumerate(cm):
        if label_idx == 0:
            continue # We don't care about the label "O"
        tp, fp, fn = matrix[1, 1], matrix[0, 1], matrix[1, 0]
        precision = divide(tp, tp + fp)
        recall = divide(tp, tp + fn)
        f1 = divide(2 * precision * recall, precision + recall)
        metrics[f"f1_{id2label[label_idx]}"] = f1
        
    f1_micro = sum(list(metrics.values())) / (n_labels - 1)
    metrics["f1_micro"] = f1_micro
        
    return metrics

In [28]:
from transformers import BertModel, BertPreTrainedModel
from transformers import RobertaPreTrainedModel, RobertaModel
from transformers.utils import (
    add_code_sample_docstrings,
    add_start_docstrings,
    add_start_docstrings_to_model_forward,
    logging,
    replace_return_docstrings,
)

from transformers.models.roberta.modeling_roberta import (
    ROBERTA_INPUTS_DOCSTRING,
    ROBERTA_START_DOCSTRING,
    RobertaEmbeddings
)

from transformers.models.bert.modeling_bert import (
    BERT_INPUTS_DOCSTRING,
    BERT_START_DOCSTRING,
    BertEmbeddings
)
from typing import Optional, Union, Tuple
from transformers.modeling_outputs import TokenClassifierOutput
from torch import nn

In [29]:
class BertForSpanCategorization(RobertaPreTrainedModel):
    _keys_to_ignore_on_load_unexpected = [r"pooler"]
    _keys_to_ignore_on_load_missing = [r"position_ids"]
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        classifier_dropout = (
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = nn.Dropout(classifier_dropout)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # Initialize weights and apply final processing
        self.post_init()
    
    @add_start_docstrings_to_model_forward(ROBERTA_INPUTS_DOCSTRING.format("batch_size, sequence_length"))
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], TokenClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size, sequence_length)`, *optional*):
            Labels for computing the token classification loss. Indices should be in `[0, ..., config.num_labels - 1]`.
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            loss = loss_fct(logits, labels.float())
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [30]:
def create_model_OTE():
    model_path = "uklfr/gottbert-base" # "uklfr/gottbert-base" # "deepset/gbert-large"
    return BertForSpanCategorization.from_pretrained(model_path, id2label=id2label, label2id=label2id)

In [31]:
trainer = Trainer(
    model_init=create_model_OTE, 
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_OTE
)

Some weights of the model checkpoint at uklfr/gottbert-base were not used when initializing BertForSpanCategorization: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing BertForSpanCategorization from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSpanCategorization from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSpanCategorization were not initialized from the model checkpoint at uklfr/gottbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

In [32]:
trainer.train()

Some weights of the model checkpoint at uklfr/gottbert-base were not used when initializing BertForSpanCategorization: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing BertForSpanCategorization from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSpanCategorization from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSpanCategorization were not initialized from the model checkpoint at uklfr/gottbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-

Epoch,Training Loss,Validation Loss,F1 B,F1 I,F1 Micro
1,0.216200,0.088065,0,0,0.000000


TrainOutput(global_step=32, training_loss=0.21620553731918335, metrics={'train_runtime': 171.7505, 'train_samples_per_second': 2.911, 'train_steps_per_second': 0.186, 'total_flos': 65324779776000.0, 'train_loss': 0.21620553731918335, 'epoch': 1.0})

### Save Results

In [33]:
results=[{}]

In [34]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

In [35]:
df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [36]:
results

[{}]